In [2]:
library(tidyverse)
library(reshape2)
library(ggpubr)
library(viridis)
library(car)
library(caret)
library(rstatix)

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: viridisLite

Loading required package: carData



In [7]:
data <- read.csv("../41467_2025_59204_MOESM4_ESM/Supplementary Software/stocks.csv" , fileEncoding="latin1")
dim(data)

[1] 3244   41

In [8]:
data_pred <- data %>% 
             mutate(Pred.30cm.Corg = case_when(!is.na(Corg.stock.30cm)  ~ Corg.stock.30cm,
                                               !is.na(Corg.stock.15cm)  ~ 1.9323329*Corg.stock.15cm-0.0010214*Corg.stock.15cm^2,
                                               Sampling.depth > 4 & is.na(Corg.stock.15cm) ~ (Corg.stock*(15/Sampling.depth)*1.9323329-0.0010214*(Corg.stock*(15/Sampling.depth)^2)) ),
                     Pred.1m.Corg = case_when(!is.na(Corg.stock.100cm)  ~ Corg.stock.100cm,
                                              !is.na(Corg.stock.50cm)  ~ 1.91*Corg.stock.50cm-0.000499*Corg.stock.50cm^2,
                                              !is.na(Corg.stock.30cm)  ~ 3.127*Corg.stock.30cm-0.00149*Corg.stock.30cm^2,
                                              !is.na(Corg.stock.15cm)  ~ 6.058*Corg.stock.15cm-0.00521*Corg.stock.15cm^2))
dim(data_pred)

[1] 3244   43

In [278]:
# Create a flag to indicate which values were predicted vs measured
data_pred <-  data_pred %>% 
              mutate(Pred.flag.30 = case_when(!is.na(Corg.stock.30cm)  ~ 'Measured 30cm', 
                                              !is.na(Corg.stock.15cm)  ~ 'Pred. from 15cm',
                                              Sampling.depth > 4 & is.na(Corg.stock.15cm) ~ 'Pred. from 5-14cm'),
                     Pred.flag.100 = case_when(!is.na(Corg.stock.100cm) ~ 'Measured 1m', 
                                              !is.na(Corg.stock.50cm)  ~ 'Pred. from 50cm',
                                              !is.na(Corg.stock.30cm)  ~ 'Pred. from 30cm',
                                              !is.na(Corg.stock.15cm)  ~ 'Pred. from 15cm')) |>
               mutate(row_id = row_number())

In [220]:
(pred_veg <- dplyr::filter(data_pred, Seagrass.functionalmorphological.group != 'Unvegetated')) |> dim()

[1] 2813   46

In [235]:
(Krause_measured_30cm <- dplyr::filter(pred_veg, Pred.flag.30 == "Measured 30cm") |>
                         dplyr::mutate(data_type = "Measured 30cm")
) |> dim()

[1] 1022   47

In [192]:
`%nin%` <- Negate(`%in%`)

In [227]:
(pred_veg_rem <- pred_veg |>
                dplyr::filter(row_id %nin% Krause_measured_30cm$row_id)) |> dim()

[1] 1791   46

In [289]:
(Krause_pred5_29cm <- pred_veg_rem |> 
                      dplyr::filter(Pred.flag.30 ==  "Pred. from 15cm" | Pred.flag.30 ==  "Pred. from 5-14cm") |>
                      dplyr::filter(Sampling.depth > 4) |>
                      dplyr::mutate(data_type = "Predicted from > 5 < 14 <= 15 cm")) |> dim()

[1] 1153   47

In [274]:
(pred_veg_rem2 <- dplyr::filter(pred_veg_rem, row_id %nin% Krause_pred5_29cm$row_id)) |> dim()

[1] 638  46

In [281]:
(Krause_less_4cm <- pred_veg_rem2 |>
                    dplyr::filter(Sampling.depth <= 4 | is.na(Sampling.depth)) |>
                    dplyr::mutate(data_type = "Predicted from < 4 cm")) |> dim()

[1] 638  47

In [293]:
Krause_database <- rbind(Krause_measured_30cm, Krause_pred5_29cm, Krause_less_4cm) 
dim(Krause_database)

[1] 2813   47

In [297]:
write_rds(Krause_database, "../data/Krause_database.rds")